In [1]:
import requests
import json

- Este notebook es para comparar los resultados del verbo *countsTaxonsGroup* con el nuevo verbo *countsTaxonsGroupGivenPoints*.

- El verbo *countsTaxonsGroup* es el que actualmente provee del análisis de nicho a la plataforma SPECIES.

* El verbo *countsTaxonsGroupGivenPoints* hace el mismo análisis que *countsTaxonsGroup* pero con puntos dados por el usuario.


In [2]:
url = 'http://species.conabio.gob.mx/api/dev/niche/'

In [3]:
req = {
  "date": True,
  "lim_inf": 1600,
  "lim_sup": 2020,
  "with_data_freq": True,
  "excluded_cells": [],
  "fosil": True,
  "idtabla": "",
  "idtime": 1557853095248,
  "apriori": False,
  "mapa_prob": False,
  "min_cells": 5,
  "with_data_score_cell": True,
  "with_data_freq_cell": True,
  "get_grid_specie": False,
  "region": 1,
  "grid_resolution": 16,
  "with_data_score_decil": True,
  "target_name": "targetGroup",
  "longitud": 0,
  "latitud": 0,
  "target_taxons": [
    {
      "taxon_rank": "species",
      "value": "Lynx rufus"
    }
  ],
  "covariables": [
    {
      "name": "GpoBio1",
      "biotic": True,
      "merge_vars": [
        {
          "rank": "order",
          "value": "Didelphimorphia",
          "level": "species"
        }
      ]
    }
  ]
}

In [4]:
req_gp = {
  "date": True,
  "lim_inf": 1600,
  "lim_sup": 2020,
  "with_data_freq": True,
  "idtabla": "",
  "fosil": True,
  "apriori": False,
  "mapa_prob": False,
  "min_cells": 5,
  "with_data_score_cell": True,
  "with_data_freq_cell": True,
  "get_grid_specie": False,
  "region": 1,
  "grid_resolution": 16,
  "with_data_score_decil": True,
  "target_name": "testing_Lynx_rufus",
  "longitud": 0,
  "latitud": 0,
  "target_points": [],
  "covariables": [
    {
      "name": "GpoBio1",
      "biotic": True,
      "merge_vars": [
        {
          "rank": "order",
          "value": "Didelphimorphia",
          "level": "species"
        }
      ]
    }
  ]
}

## Se leen los datos

In [5]:
#file = './data/Aedes_aegypti.json'
file = './data/Lynx_rufus.json'
#file = './data/Panthera_onca.json'
#file = './data/Kricogonia_lyside.json'
#file = './data/Achillea_millefolium.json'
#file = './data/Trifolium_repens.json'

with open(file, 'r') as f:
    
    target_points = json.loads(str(f.read()))['data']
    
    
req_gp['target_points'] = target_points

## Comparación

In [6]:
res = requests.post(url + 'countsTaxonsGroup', json=req).json()

for obj in res['data']:
    print(obj['generovalido'] + ' ' + obj['especieepiteto'], len(obj['cells_map']))

Didelphis virginiana 654
Tlacuatzin canescens 151
Didelphis marsupialis 273
Philander opossum 206
Marmosa mexicana 125
Caluromys derbianus 31
Marmosa mayensis 9
Chironectes minimus 14


In [7]:
res_gp = requests.post(url + 'countsTaxonsGroupGivenPoints', json=req_gp).json()

for obj in res_gp['data']:
    print(obj['generovalido'] + ' ' + obj['especieepiteto'], len(obj['cells_map']))

Didelphis virginiana 654
Tlacuatzin canescens 151
Didelphis marsupialis 273
Philander opossum 206
Marmosa mexicana 125
Caluromys derbianus 31
Marmosa mayensis 9
Chironectes minimus 14


In [8]:
assert res.keys() == res_gp.keys(), f"{res.keys()} \n {res_gp.keys()}"

In [9]:
assert len(res['data']) == len(res_gp['data'])

In [10]:
def get_species_from_group(data, group_name, genus, epithet):    
    for elem in data:
        if elem['group_name'] == group_name:
            if (elem['generovalido'] == genus) and (elem['especieepiteto'] == epithet):
                species_data = elem
                return species_data
    
    return None
            

In [ ]:
get_species_from_group(res_gp['data'], 'GpoBio1', 'Tlacuatzin', 'canescens')

In [12]:
for i, obj in enumerate(res['data']):
    obj_gp = get_species_from_group(res_gp['data'], 'GpoBio1', obj['generovalido'], obj['especieepiteto'])
    
    assert obj.keys() == obj_gp.keys()
    
    for k, v in obj.items():
        if k == 'cells':
            assert set(obj_gp[k]) == set(v), f"Data element: {i} Key: {k} \n{100*'-'}\n {set(obj_gp[k]).difference(v)}"
        else:
            assert obj_gp[k] == v, f"Data info: {obj['generovalido']} {obj['especieepiteto']} {i} Key: {k} \n{100*'-'}\n GP: {obj_gp[k]} Orig: {v}"


## Con validación?

In [13]:
req_gp = {
  "date": True,
  "lim_inf": 1600,
  "lim_sup": 2020,
  "with_data_freq": True,
  "idtabla": "",
  "fosil": True,
  "apriori": False,
  "mapa_prob": False,
  "min_cells": 5,
  "with_data_score_cell": True,
  "with_data_freq_cell": True,
  "get_grid_specie": False,
  "region": 1,
  "grid_resolution": 16,
  "with_data_score_decil": True,
  "target_name": "testing_Lynx_rufus",
  "longitud": 0,
  "latitud": 0,
  "iterations": 5,
  "target_points": [],
  "covariables": [
    {
      "name": "GpoBio1",
      "biotic": True,
      "merge_vars": [
        {
          "rank": "order",
          "value": "Didelphimorphia",
          "level": "species"
        }
      ]
    }
  ]
}

In [14]:
res = requests.post(url + '/especie/getGroupValidationTables', json=req).json()
validation_table = res['data'][0]['tblname']

res_gp = requests.post(url + '/especie/getGivenPointaValidationTables', json=req_gp).json()
validation_table_gp = res['data'][0]['tblname']


req['idtabla'] = validation_table
req_gp['idtabla'] = validation_table_gp